In [15]:
# Importar las librerías necesarias
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU

In [ ]:
### DUDAS

# tenemos que usar convolusionales o LSTM? 
#que es el dense? 
# agregar features?
# tenemos que normalizar?
# nuestro input es el window size para el LSTM y la cantidad de caracteristicas? diferencia entre las convolusionales 2D?
# si tenemos 2000 datos por feature, cuantas neuronas tenemos de input? 2000? nfeatures segun yo o mas bien, como relaciono el input shape con las neuronas?
# segun yo cada feature representaba una neurona para la capa inicial, o es cada dato, o es independiente?
# lo anterior como sería con convolusionales? los pixeles en convolusionales son features? explicar el por el video que vi
# como sabemos que capa despues de la anterior, es decir LSTM y luego dropout o leaky relu o que onda
# en que parte se cambia la funcion de costo? es decir cual es la diferencia entre como planteamos BinaryCrossentropy en GAN normal?
# Para que sirve import tqdm?
# Tarda mucho en correr?



In [2]:
# Separar en train y test el data set


def download_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data = data[['Adj Close']]  
    return data


def preprocess_data(data: pd.DataFrame):
    data = data.pct_change().dropna()
    mean = data.mean()
    std = data.std()
    data_norm = (data - mean)/std 
    
    return data, data_norm



input_shape = (50, 3)  # 50 pasos en el tiempo (ventana), 3 features

# Crear el modelo
model = Sequential()
model.add(LSTM(64, input_shape=input_shape, return_sequences=True))


# Definir el generador
def build_generator(latent_dim, seq_len, features):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(seq_len, latent_dim)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(LSTM(100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(features))  # salida con el mismo número de características que la serie temporal

    return model

def build_discriminator(seq_len, features):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(seq_len, features)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(LSTM(100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))  # salida binaria para clasificar real o falso

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [3]:
ticker = "MANU"  
start_date = '2014-10-29'
end_date = '2024-10-30'
data = download_data(ticker, start_date, end_date)
data, data_norm = preprocess_data(data)

[*********************100%%**********************]  1 of 1 completed


In [4]:
data_norm.head()

,Adj Close
Date,
2014-10-30,-0.203636
2014-10-31,0.042260
2014-11-03,0.233881
2014-11-04,-0.720528
2014-11-05,-0.233994


In [ ]:
#A como lo entiendo esto se mantendría igual lo que cambia del promedio de la resta es mas adelante
loss_function = tf.keras.losses.BinaryCrossentropy(from_logits = True)

In [ ]:
def discriminator_loss(y_true, y_fake):
    real_loss = loss_function(tf.ones_like(y_true), y_true)
    fake_loss = loss_function(tf.zeros_like(y_fake), y_fake)
    return real_loss + fake_loss


def generator_loss(y_fake):
    return loss_function(tf.ones_like(y_fake), y_fake)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
@tf.function
def train_step(images, batch_size = 200):
    noise = tf.random.normal([batch_size, 100])
    
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = gen_model(noise, training = True)
        
        y_real = disc_model(images, training = True)
        y_fake = disc_model(generated_images, training = True)
        
        gen_loss = generator_loss(-tf.math.reduce_mean(y_fake)) # o simplemente -tf.math.reduce_mean(y_fake) y sin las funciones de gen_loss y disc_loss
        disc_loss = discriminator_loss(tf.reduce_mean(y_fake) - tf.reduce_mean(y_real)) #o simplemente tf.reduce_mean(y_fake) - tf.reduce_mean(y_real)
        
        
    gradients_gen = gen_tape.gradient(gen_loss, gen_model.trainable_variables)
    gradients_disc = disc_tape.gradient(disc_loss, disc_model.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_gen, gen_model.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_disc, disc_model.trainable_variables))

In [ ]:
for epoch in range(5):
    for i in tqdm.tqdm(range(num_batches)):
        batch = x_train_norm[i*200:(1+i)*200]
        train_step(batch)

In [12]:
# Instanciar el generador y el critic
critic_model = critic()
gen_model = generator()

# Definir optimizadores
generator_optimizer = tf.keras.optimizers.RMSprop(1e-4)
critic_optimizer = tf.keras.optimizers.RMSprop(1e-4)

# Clipping constante para los pesos del critic
weight_clip = 0.01

@tf.function
def train_step(real_data, batch_size=64):
    noise = tf.random.normal([batch_size, 100])

    # Entrenar el critic varias veces por cada paso del generador
    for _ in range(5):
        with tf.GradientTape() as disc_tape:
            generated_sequences = gen_model(noise, training=True)
            y_real = critic_model(real_data, training=True)
            y_fake = critic_model(generated_sequences, training=True)
            # Loss del critic basada en la distancia Wasserstein
            disc_loss = tf.reduce_mean(y_fake) - tf.reduce_mean(y_real)
        
        gradients_disc = disc_tape.gradient(disc_loss, critic_model.trainable_variables)
        critic_optimizer.apply_gradients(zip(gradients_disc, critic_model.trainable_variables))
        
        # Clipping de pesos para mantener la función 1-Lipschitz
        for var in critic_model.trainable_variables:
            var.assign(tf.clip_by_value(var, -weight_clip, weight_clip))

    # Entrenar el generador
    with tf.GradientTape() as gen_tape:
        generated_sequences = gen_model(noise, training=True)
        y_fake = critic_model(generated_sequences, training=True)
        # Loss del generador, que intenta maximizar la salida del critic
        gen_loss = -tf.reduce_mean(y_fake)
        
    gradients_gen = gen_tape.gradient(gen_loss, gen_model.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_gen, gen_model.trainable_variables))

# Entrenar el modelo WGAN
def train_wgan(train_data, epochs=1, batch_size=64):
    for epoch in range(epochs):
        for i in range(len(train_data) // batch_size):
            batch = train_data[i*batch_size:(i+1)*batch_size]
            batch = np.expand_dims(batch, -1)  # Añadir dimensión para compatibilidad con el modelo
            train_step(batch)
        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{epochs} completado.")

# Entrenar el WGAN
train_wgan(data_norm, epochs=1, batch_size=64)

# Generar escenarios de prueba con el generador entrenado
def generate_scenarios(gen_model, num_scenarios=100, sequence_length=50):
    noise = tf.random.normal([num_scenarios, 100])
    generated_sequences = gen_model(noise, training=False)
    generated_sequences = generated_sequences.numpy()
    # Desnormalizar los datos
    generated_sequences = (generated_sequences + 1) / 2  # Cambiar de rango [-1, 1] a [0, 1]
    generated_sequences = generated_sequences * (data['Close'].max() - data['Close'].min()) + data['Close'].min()
    return generated_sequences

# Generar 100 escenarios de prueba
scenarios = generate_scenarios(gen_model, num_scenarios=100, sequence_length=50)

# Visualizar algunos de los escenarios generados
plt.figure(figsize=(12, 6))
for i in range(5):  # Mostrar 5 ejemplos
    plt.plot(scenarios[i], label=f'Scenario {i+1}')
plt.title("Escenarios de Precios Generados por el WGAN para TSLA")
plt.xlabel("Días")
plt.ylabel("Precio")
plt.legend()
plt.show()



Epoch 0/1 completado.


KeyError: 'Close'

In [6]:
# Ejemplo de Estrategia Simple para Backtesting
def simple_trading_strategy(prices, stop_loss=0.05, take_profit=0.1):
    cash = 10000  # Capital inicial
    position = 0  # No tenemos posición al inicio
    portfolio_value = [cash]  # Valor del portafolio en el tiempo

    for i in range(1, len(prices)):
        if position == 0 and prices[i] > prices[i - 1]:  # Señal de compra
            position = cash / prices[i]
            cash = 0
        elif position > 0:
            # Calcular P&L en porcentaje
            pnl = (prices[i] - prices[i - 1]) / prices[i - 1]
            if pnl <= -stop_loss or pnl >= take_profit:  # Condiciones de venta
                cash = position * prices[i]
                position = 0
        # Actualizar el valor del portafolio
        portfolio_value.append(cash + position * prices[i])

    return portfolio_value

# Backtesting de la estrategia en los escenarios generados
all_portfolio_values = []
for scenario in scenarios:
    portfolio_values = simple_trading_strategy(scenario)
    all_portfolio_values.append(portfolio_values)

# Visualizar el rendimiento de la estrategia en los escenarios
plt.figure(figsize=(12, 6))
for i in range(5):  # Mostrar 5 ejemplos de rendimiento
    plt.plot(all_portfolio_values[i], label=f'Scenario {i+1}')
plt.title("Rendimiento de la Estrategia en Escenarios Generados para TSLA")
plt.xlabel("Días")
plt.ylabel("Valor del Portafolio")
plt.legend()
plt.show()


NameError: name 'scenarios' is not defined